# Restructuring Data into a Tidy Form

In [1]:
import pandas as pd
import numpy as np

### Tidying variable values as column names with stack

In [2]:
state_fruit = pd.read_csv('data/state_fruit.csv', index_col=0)
state_fruit

Apple  Orange  Banana
Texas       12      10      40
Arizona      9       7      12
Florida      0      14     190

In [3]:
state_fruit.stack()

Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [4]:
state_fruit_tidy = state_fruit.stack().reset_index()
state_fruit_tidy

level_0 level_1    0
0    Texas   Apple   12
1    Texas  Orange   10
2    Texas  Banana   40
3  Arizona   Apple    9
4  Arizona  Orange    7
5  Arizona  Banana   12
6  Florida   Apple    0
7  Florida  Orange   14
8  Florida  Banana  190

In [5]:
state_fruit_tidy.columns = ['state', 'fruit', 'weight']
state_fruit_tidy

state   fruit  weight
0    Texas   Apple      12
1    Texas  Orange      10
2    Texas  Banana      40
3  Arizona   Apple       9
4  Arizona  Orange       7
5  Arizona  Banana      12
6  Florida   Apple       0
7  Florida  Orange      14
8  Florida  Banana     190

In [6]:
state_fruit.stack()\
           .rename_axis(['state', 'fruit'])

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [7]:
state_fruit.stack()\
           .rename_axis(['state', 'fruit'])\
           .reset_index(name='weight')

state   fruit  weight
0    Texas   Apple      12
1    Texas  Orange      10
2    Texas  Banana      40
3  Arizona   Apple       9
4  Arizona  Orange       7
5  Arizona  Banana      12
6  Florida   Apple       0
7  Florida  Orange      14
8  Florida  Banana     190

### Tidying variable values as column names with melt

In [8]:
state_fruit2 = pd.read_csv('data/state_fruit2.csv')
state_fruit2

State  Apple  Orange  Banana
0    Texas     12      10      40
1  Arizona      9       7      12
2  Florida      0      14     190

In [9]:
state_fruit2.melt(id_vars=['State'],
                 value_vars=['Apple', 'Orange', 'Banana'])

State variable  value
0    Texas    Apple     12
1  Arizona    Apple      9
2  Florida    Apple      0
3    Texas   Orange     10
4  Arizona   Orange      7
5  Florida   Orange     14
6    Texas   Banana     40
7  Arizona   Banana     12
8  Florida   Banana    190

In [10]:
state_fruit2.melt(id_vars=['State'],
                 value_vars=['Apple', 'Orange', 'Banana'],
                 var_name='Fruit',
                 value_name='Weight')

State   Fruit  Weight
0    Texas   Apple      12
1  Arizona   Apple       9
2  Florida   Apple       0
3    Texas  Orange      10
4  Arizona  Orange       7
5  Florida  Orange      14
6    Texas  Banana      40
7  Arizona  Banana      12
8  Florida  Banana     190

In [11]:
state_fruit2.melt()

variable    value
0     State    Texas
1     State  Arizona
2     State  Florida
3     Apple       12
4     Apple        9
5     Apple        0
6    Orange       10
7    Orange        7
8    Orange       14
9    Banana       40
10   Banana       12
11   Banana      190

In [12]:
state_fruit2.melt(id_vars='State')

State variable  value
0    Texas    Apple     12
1  Arizona    Apple      9
2  Florida    Apple      0
3    Texas   Orange     10
4  Arizona   Orange      7
5  Florida   Orange     14
6    Texas   Banana     40
7  Arizona   Banana     12
8  Florida   Banana    190

### Stacking multiple groups of variables simultaneously

In [13]:
movie = pd.read_csv('data/movie.csv')
actor = movie[['movie_title', 'actor_1_name', 'actor_2_name', 'actor_3_name', 
               'actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes']]
actor.head()

movie_title     actor_1_name  \
0                                      Avatar      CCH Pounder   
1    Pirates of the Caribbean: At World's End      Johnny Depp   
2                                     Spectre  Christoph Waltz   
3                       The Dark Knight Rises        Tom Hardy   
4  Star Wars: Episode VII - The Force Awakens      Doug Walker   

       actor_2_name          actor_3_name  actor_1_facebook_likes  \
0  Joel David Moore             Wes Studi                  1000.0   
1     Orlando Bloom        Jack Davenport                 40000.0   
2      Rory Kinnear      Stephanie Sigman                 11000.0   
3    Christian Bale  Joseph Gordon-Levitt                 27000.0   
4        Rob Walker                   NaN                   131.0   

   actor_2_facebook_likes  actor_3_facebook_likes  
0                   936.0                   855.0  
1                  5000.0                  1000.0  
2                   393.0                   161.0  
3                 23000.0                 23000.0  
4                    12.0                     NaN

In [14]:
def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = col_name[:5] + col_name[fb_idx - 1:] + col_name[5:fb_idx-1]
    return col_name

In [15]:
actor2 = actor.rename(columns=change_col_name)
actor2.head()

movie_title          actor_1  \
0                                      Avatar      CCH Pounder   
1    Pirates of the Caribbean: At World's End      Johnny Depp   
2                                     Spectre  Christoph Waltz   
3                       The Dark Knight Rises        Tom Hardy   
4  Star Wars: Episode VII - The Force Awakens      Doug Walker   

            actor_2               actor_3  actor_facebook_likes_1  \
0  Joel David Moore             Wes Studi                  1000.0   
1     Orlando Bloom        Jack Davenport                 40000.0   
2      Rory Kinnear      Stephanie Sigman                 11000.0   
3    Christian Bale  Joseph Gordon-Levitt                 27000.0   
4        Rob Walker                   NaN                   131.0   

   actor_facebook_likes_2  actor_facebook_likes_3  
0                   936.0                   855.0  
1                  5000.0                  1000.0  
2                   393.0                   161.0  
3                 23000.0                 23000.0  
4                    12.0                     NaN

In [16]:
stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = pd.wide_to_long(actor2, 
                              stubnames=stubs, 
                              i=['movie_title'], 
                              j='actor_num', 
                              sep='_').reset_index()
actor2_tidy.head()

movie_title actor_num            actor  \
0                                      Avatar         1      CCH Pounder   
1    Pirates of the Caribbean: At World's End         1      Johnny Depp   
2                                     Spectre         1  Christoph Waltz   
3                       The Dark Knight Rises         1        Tom Hardy   
4  Star Wars: Episode VII - The Force Awakens         1      Doug Walker   

   actor_facebook_likes  
0                1000.0  
1               40000.0  
2               11000.0  
3               27000.0  
4                 131.0

In [17]:
df = pd.read_csv('data/stackme.csv')
df

State Country    a1   b2   Test  d  e
0    TX      US  0.45  0.3  Test1  2  6
1    MA      US  0.03  1.2  Test2  9  7
2    ON     CAN  0.70  4.2  Test3  4  2

In [18]:
df2 = df.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2',
                           'd':'group2_a1', 'e':'group2_b2'})
df2

State Country  group1_a1  group1_b2   Test  group2_a1  group2_b2
0    TX      US       0.45        0.3  Test1          2          6
1    MA      US       0.03        1.2  Test2          9          7
2    ON     CAN       0.70        4.2  Test3          4          2

In [19]:
pd.wide_to_long(df2, 
                stubnames=['group1', 'group2'], 
                i=['State', 'Country', 'Test'], 
                j='Label', 
                suffix='.+', 
                sep='_')

group1  group2
State Country Test  Label                
TX    US      Test1 a1       0.45       2
                    b2       0.30       6
MA    US      Test2 a1       0.03       9
                    b2       1.20       7
ON    CAN     Test3 a1       0.70       4
                    b2       4.20       2

### Inverting stacked data

In [20]:
usecol_func = lambda x: 'UGDS_' in x or x == 'INSTNM'
college = pd.read_csv('data/college.csv', 
                          index_col='INSTNM', 
                          usecols=usecol_func)
college.head()

UGDS_WHITE  UGDS_BLACK  UGDS_HISP  \
INSTNM                                                                   
Alabama A & M University                 0.0333      0.9353     0.0055   
University of Alabama at Birmingham      0.5922      0.2600     0.0283   
Amridge University                       0.2990      0.4192     0.0069   
University of Alabama in Huntsville      0.6988      0.1255     0.0382   
Alabama State University                 0.0158      0.9208     0.0121   

                                     UGDS_ASIAN  UGDS_AIAN  UGDS_NHPI  \
INSTNM                                                                  
Alabama A & M University                 0.0019     0.0024     0.0019   
University of Alabama at Birmingham      0.0518     0.0022     0.0007   
Amridge University                       0.0034     0.0000     0.0000   
University of Alabama in Huntsville      0.0376     0.0143     0.0002   
Alabama State University                 0.0019     0.0010     0.0006   

                                     UGDS_2MOR  UGDS_NRA  UGDS_UNKN  
INSTNM                                                               
Alabama A & M University                0.0000    0.0059     0.0138  
University of Alabama at Birmingham     0.0368    0.0179     0.0100  
Amridge University                      0.0000    0.0000     0.2715  
University of Alabama in Huntsville     0.0172    0.0332     0.0350  
Alabama State University                0.0098    0.0243     0.0137

In [21]:
college_stacked = college.stack()
college_stacked.head(18)

INSTNM                                         
Alabama A & M University             UGDS_WHITE    0.0333
                                     UGDS_BLACK    0.9353
                                     UGDS_HISP     0.0055
                                     UGDS_ASIAN    0.0019
                                     UGDS_AIAN     0.0024
                                     UGDS_NHPI     0.0019
                                     UGDS_2MOR     0.0000
                                     UGDS_NRA      0.0059
                                     UGDS_UNKN     0.0138
University of Alabama at Birmingham  UGDS_WHITE    0.5922
                                     UGDS_BLACK    0.2600
                                     UGDS_HISP     0.0283
                                     UGDS_ASIAN    0.0518
                                     UGDS_AIAN     0.0022
                                     UGDS_NHPI     0.0007
                                     UGDS_2MOR     0.0368
                        

In [22]:
college_stacked.unstack()

UGDS_WHITE  UGDS_BLACK  \
INSTNM                                                                       
Alabama A & M University                                0.0333      0.9353   
University of Alabama at Birmingham                     0.5922      0.2600   
Amridge University                                      0.2990      0.4192   
University of Alabama in Huntsville                     0.6988      0.1255   
Alabama State University                                0.0158      0.9208   
The University of Alabama                               0.7825      0.1119   
Central Alabama Community College                       0.7255      0.2613   
Athens State University                                 0.7823      0.1200   
Auburn University at Montgomery                         0.5328      0.3376   
Auburn University                                       0.8507      0.0704   
Birmingham Southern College                             0.7983      0.1102   
Chattahoochee Valley Community College                  0.4661      0.4372   
Concordia College Alabama                               0.0280      0.8758   
South University-Montgomery                             0.3046      0.6054   
Enterprise State Community College                      0.6408      0.2435   
James H Faulkner State Community College                0.6979      0.2259   
Faulkner University                                     0.3874      0.5137   
Gadsden State Community College                         0.6921      0.2076   
New Beginning College of Cosmetology                    0.8957      0.0348   
George C Wallace State Community College-Dothan         0.6273      0.3240   
George C Wallace State Community College-Hancev...      0.8630      0.0612   
George C Wallace State Community College-Selma          0.1956      0.7449   
Herzing University-Birmingham                           0.3543      0.5265   
Huntingdon College                                      0.6388      0.1993   
Heritage Christian University                           0.7419      0.1129   
J F Drake State Community and Technical College         0.3337      0.5965   
Jacksonville State University                           0.6834      0.2382   
Jefferson Davis Community College                       0.5857      0.2982   
Jefferson State Community College                       0.6696      0.2246   
John C Calhoun State Community College                  0.6528      0.1940   
...                                                        ...         ...   
Tint School of Makeup & Cosmetology-Seattle             0.5368      0.0737   
Centro de Estudios Multidisciplinarios-Mayaguez         0.0000      0.0000   
Heritage College-Cleveland                              0.3469      0.5102   
Aveda Institute-Phoenix                                 0.6364      0.0341   
University of North Texas at Dallas                     0.1319      0.3358   
BridgeValley Community & Technical College              0.8967      0.0753   
Northwest College-Medford                               0.7000      0.0000   
Virginia College-Fort Pierce                            0.2565      0.5696   
Houston Training Schools-Gessner                        0.0444      0.1222   
Empire Beauty School-Buffalo                            0.4906      0.3585   
Paul Mitchell the School-Toledo                         0.4211      0.2719   
MJ's Barber Academy                                     0.0000      0.9333   
Trine University-Arizona Regional Campus                0.5000      0.0714   
Regency Beauty Institute-Gaithersburg                   0.2250      0.3250   
Georgia Military College                                0.4011      0.4460   
New Horizons Medical Institute-Winder                   0.4175      0.5000   
SAE Institute of Technology-Chicago                     0.2797      0.4576   
Beau Monde College of Hair Design-Beau Monde Ac...      0.6400      0.0800   
Fayette Beauty Academy-Ritz Beauty Academy              0.0000      1.0000

In [23]:
college2 = pd.read_csv('data/college.csv', 
                      usecols=usecol_func)
college2.head()

INSTNM  UGDS_WHITE  UGDS_BLACK  UGDS_HISP  \
0             Alabama A & M University      0.0333      0.9353     0.0055   
1  University of Alabama at Birmingham      0.5922      0.2600     0.0283   
2                   Amridge University      0.2990      0.4192     0.0069   
3  University of Alabama in Huntsville      0.6988      0.1255     0.0382   
4             Alabama State University      0.0158      0.9208     0.0121   

   UGDS_ASIAN  UGDS_AIAN  UGDS_NHPI  UGDS_2MOR  UGDS_NRA  UGDS_UNKN  
0      0.0019     0.0024     0.0019     0.0000    0.0059     0.0138  
1      0.0518     0.0022     0.0007     0.0368    0.0179     0.0100  
2      0.0034     0.0000     0.0000     0.0000    0.0000     0.2715  
3      0.0376     0.0143     0.0002     0.0172    0.0332     0.0350  
4      0.0019     0.0010     0.0006     0.0098    0.0243     0.0137

In [24]:
college_melted = college2.melt(id_vars='INSTNM', 
                               var_name='Race',
                               value_name='Percentage')
college_melted.head()

INSTNM        Race  Percentage
0             Alabama A & M University  UGDS_WHITE      0.0333
1  University of Alabama at Birmingham  UGDS_WHITE      0.5922
2                   Amridge University  UGDS_WHITE      0.2990
3  University of Alabama in Huntsville  UGDS_WHITE      0.6988
4             Alabama State University  UGDS_WHITE      0.0158

In [25]:
melted_inv = college_melted.pivot(index='INSTNM',
                                  columns='Race',
                                  values='Percentage')
melted_inv.head()

Race                                                UGDS_2MOR  UGDS_AIAN  \
INSTNM                                                                     
A & W Healthcare Educators                             0.0000        0.0   
A T Still University of Health Sciences                   NaN        NaN   
ABC Beauty Academy                                     0.0000        0.0   
ABC Beauty College Inc                                 0.0000        0.0   
AI Miami International University of Art and De...     0.0018        0.0   

Race                                                UGDS_ASIAN  UGDS_BLACK  \
INSTNM                                                                       
A & W Healthcare Educators                              0.0000      0.9750   
A T Still University of Health Sciences                    NaN         NaN   
ABC Beauty Academy                                      0.9333      0.0333   
ABC Beauty College Inc                                  0.0000      0.6579   
AI Miami International University of Art and De...      0.0018      0.0198   

Race                                                UGDS_HISP  UGDS_NHPI  \
INSTNM                                                                     
A & W Healthcare Educators                             0.0250        0.0   
A T Still University of Health Sciences                   NaN        NaN   
ABC Beauty Academy                                     0.0333        0.0   
ABC Beauty College Inc                                 0.0526        0.0   
AI Miami International University of Art and De...     0.4773        0.0   

Race                                                UGDS_NRA  UGDS_UNKN  \
INSTNM                                                                    
A & W Healthcare Educators                            0.0000     0.0000   
A T Still University of Health Sciences                  NaN        NaN   
ABC Beauty Academy                                    0.0000     0.0000   
ABC Beauty College Inc                                0.0000     0.0000   
AI Miami International University of Art and De...    0.0025     0.4644   

Race                                                UGDS_WHITE  
INSTNM                                                          
A & W Healthcare Educators                              0.0000  
A T Still University of Health Sciences                    NaN  
ABC Beauty Academy                                      0.0000  
ABC Beauty College Inc                                  0.2895  
AI Miami International University of Art and De...      0.0324

In [26]:
college2_replication = melted_inv.loc[college2['INSTNM'], 
                                      college2.columns[1:]]\
                                         .reset_index()
college2.equals(college2_replication)

True

### Unstacking after a groupby aggregation

In [27]:
employee = pd.read_csv('data/employee.csv')

In [28]:
employee.groupby('RACE')['BASE_SALARY'].mean().astype(int)

RACE
American Indian or Alaskan Native    60272
Asian/Pacific Islander               61660
Black or African American            50137
Hispanic/Latino                      52345
Others                               51278
White                                64419
Name: BASE_SALARY, dtype: int32

In [29]:
agg = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'].mean().astype(int)
agg

RACE                               GENDER
American Indian or Alaskan Native  Female    60238
                                   Male      60305
Asian/Pacific Islander             Female    63226
                                   Male      61033
Black or African American          Female    48915
                                   Male      51082
Hispanic/Latino                    Female    46503
                                   Male      54782
Others                             Female    63785
                                   Male      38771
White                              Female    66793
                                   Male      63940
Name: BASE_SALARY, dtype: int32

In [30]:
agg.unstack('GENDER')

GENDER                             Female   Male
RACE                                            
American Indian or Alaskan Native   60238  60305
Asian/Pacific Islander              63226  61033
Black or African American           48915  51082
Hispanic/Latino                     46503  54782
Others                              63785  38771
White                               66793  63940

In [31]:
agg.unstack('RACE')

RACE    American Indian or Alaskan Native  Asian/Pacific Islander  \
GENDER                                                              
Female                              60238                   63226   
Male                                60305                   61033   

RACE    Black or African American  Hispanic/Latino  Others  White  
GENDER                                                             
Female                      48915            46503   63785  66793  
Male                        51082            54782   38771  63940

### Replicating pivot_table with a groupby aggregation

In [37]:
flights = pd.read_csv('data/flights.csv')
flights.head()

MONTH  DAY  WEEKDAY AIRLINE ORG_AIR DEST_AIR  SCHED_DEP  DEP_DELAY  \
0      1    1        4      WN     LAX      SLC       1625       58.0   
1      1    1        4      UA     DEN      IAD        823        7.0   
2      1    1        4      MQ     DFW      VPS       1305       36.0   
3      1    1        4      AA     DFW      DCA       1555        7.0   
4      1    1        4      WN     LAX      MCI       1720       48.0   

   AIR_TIME  DIST  SCHED_ARR  ARR_DELAY  DIVERTED  CANCELLED  
0      94.0   590       1905       65.0         0          0  
1     154.0  1452       1333      -13.0         0          0  
2      85.0   641       1453       35.0         0          0  
3     126.0  1192       1935       -7.0         0          0  
4     166.0  1363       2225       39.0         0          0

In [38]:
fp = flights.pivot_table(index='AIRLINE', 
                         columns='ORG_AIR', 
                         values='CANCELLED', 
                         aggfunc='sum',
                         fill_value=0).round(2)
fp.head()

ORG_AIR  ATL  DEN  DFW  IAH  LAS  LAX  MSP  ORD  PHX  SFO
AIRLINE                                                  
AA         3    4   86    3    3   11    3   35    4    2
AS         0    0    0    0    0    0    0    0    0    0
B6         0    0    0    0    0    0    0    0    0    1
DL        28    1    0    0    1    1    4    0    1    2
EV        18    6   27   36    0    0    6   53    0    0

In [39]:
fg = flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()
fg.head()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
Name: CANCELLED, dtype: int64

In [40]:
fg_unstack = fg.unstack('ORG_AIR', fill_value=0)
fg_unstack.head()

ORG_AIR  ATL  DEN  DFW  IAH  LAS  LAX  MSP  ORD  PHX  SFO
AIRLINE                                                  
AA         3    4   86    3    3   11    3   35    4    2
AS         0    0    0    0    0    0    0    0    0    0
B6         0    0    0    0    0    0    0    0    0    1
DL        28    1    0    0    1    1    4    0    1    2
EV        18    6   27   36    0    0    6   53    0    0

In [41]:
fp.equals(fg_unstack)

True

### Renaming axis levels for easy reshaping

In [42]:
college = pd.read_csv('data/college.csv')
cg = college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATMTMID'] \
            .agg(['count', 'min', 'max']).head(6)
cg

UGDS                 SATMTMID              
                count    min      max    count    min    max
STABBR RELAFFIL                                             
AK     0            7  109.0  12865.0        0    NaN    NaN
       1            3   27.0    275.0        1  503.0  503.0
AL     0           71   12.0  29851.0       13  420.0  590.0
       1           18   13.0   3033.0        8  400.0  560.0
AR     0           68   18.0  21405.0        9  427.0  565.0
       1           14   20.0   4485.0        7  495.0  600.0

In [44]:
cg = cg.rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
cg

AGG_COLS         UGDS                 SATMTMID              
AGG_FUNCS       count    min      max    count    min    max
STABBR RELAFFIL                                             
AK     0            7  109.0  12865.0        0    NaN    NaN
       1            3   27.0    275.0        1  503.0  503.0
AL     0           71   12.0  29851.0       13  420.0  590.0
       1           18   13.0   3033.0        8  400.0  560.0
AR     0           68   18.0  21405.0        9  427.0  565.0
       1           14   20.0   4485.0        7  495.0  600.0

In [45]:
cg.stack('AGG_FUNCS').head()

AGG_COLS                      UGDS  SATMTMID
STABBR RELAFFIL AGG_FUNCS                   
AK     0        count          7.0       0.0
                min          109.0       NaN
                max        12865.0       NaN
       1        count          3.0       1.0
                min           27.0     503.0

In [46]:
cg.stack('AGG_FUNCS').swaplevel('AGG_FUNCS', 'STABBR', axis='index').head()

AGG_COLS                      UGDS  SATMTMID
AGG_FUNCS RELAFFIL STABBR                   
count     0        AK          7.0       0.0
min       0        AK        109.0       NaN
max       0        AK      12865.0       NaN
count     1        AK          3.0       1.0
min       1        AK         27.0     503.0

In [47]:
cg.stack('AGG_FUNCS') \
  .swaplevel('AGG_FUNCS', 'STABBR', axis='index') \
  .sort_index(level='RELAFFIL', axis='index') \
  .sort_index(level='AGG_COLS', axis='columns').head(6)

AGG_COLS                   SATMTMID   UGDS
AGG_FUNCS RELAFFIL STABBR                 
count     0        AK           0.0    7.0
                   AL          13.0   71.0
                   AR           9.0   68.0
min       0        AK           NaN  109.0
                   AL         420.0   12.0
                   AR         427.0   18.0

In [48]:
cg.stack('AGG_FUNCS').unstack(['RELAFFIL', 'STABBR'])

AGG_COLS      UGDS                                          SATMTMID         \
RELAFFIL         0      1        0       1        0       1        0      1   
STABBR          AK     AK       AL      AL       AR      AR       AK     AK   
AGG_FUNCS                                                                     
count          7.0    3.0     71.0    18.0     68.0    14.0      0.0    1.0   
min          109.0   27.0     12.0    13.0     18.0    20.0      NaN  503.0   
max        12865.0  275.0  29851.0  3033.0  21405.0  4485.0      NaN  503.0   

AGG_COLS                               
RELAFFIL       0      1      0      1  
STABBR        AL     AL     AR     AR  
AGG_FUNCS                              
count       13.0    8.0    9.0    7.0  
min        420.0  400.0  427.0  495.0  
max        590.0  560.0  565.0  600.0

In [49]:
cg.stack(['AGG_FUNCS', 'AGG_COLS']).head(12)

STABBR  RELAFFIL  AGG_FUNCS  AGG_COLS
AK      0         count      UGDS            7.0
                             SATMTMID        0.0
                  min        UGDS          109.0
                  max        UGDS        12865.0
        1         count      UGDS            3.0
                             SATMTMID        1.0
                  min        UGDS           27.0
                             SATMTMID      503.0
                  max        UGDS          275.0
                             SATMTMID      503.0
AL      0         count      UGDS           71.0
                             SATMTMID       13.0
dtype: float64

### Tidying when multiple variables are stored as column names

In [52]:
weightlifting = pd.read_csv('data/weightlifting_men.csv')
weightlifting

Weight Category  M35 35-39  M40 40-44  M45 45-49  M50 50-54  M55 55-59  \
0              56        137        130        125        115        102   
1              62        152        145        137        127        112   
2              69        167        160        150        140        125   
3              77        182        172        165        150        135   
4              85        192        182        175        160        142   
5              94        202        192        182        167        150   
6             105        210        200        190        175        157   
7            105+        217        207        197        182        165   

   M60 60-64  M65 65-69  M70 70-74  M75 75-79  M80 80+  
0         92         80         67         62       55  
1        102         90         75         67       57  
2        112         97         82         75       60  
3        122        107         90         82       65  
4        130        112         95         87       70  
5        137        120        100         90       75  
6        142        122        102         95       80  
7        150        127        107        100       85

In [53]:
wl_melt = weightlifting.melt(id_vars='Weight Category', 
                             var_name='sex_age', 
                             value_name='Qual Total')
wl_melt.head()

Weight Category    sex_age  Qual Total
0              56  M35 35-39         137
1              62  M35 35-39         152
2              69  M35 35-39         167
3              77  M35 35-39         182
4              85  M35 35-39         192

In [54]:
sex_age = wl_melt['sex_age'].str.split(expand=True)
sex_age.head()

0      1
0  M35  35-39
1  M35  35-39
2  M35  35-39
3  M35  35-39
4  M35  35-39

In [55]:
sex_age.columns = ['Sex', 'Age Group']
sex_age.head()

Sex Age Group
0  M35     35-39
1  M35     35-39
2  M35     35-39
3  M35     35-39
4  M35     35-39

In [56]:
sex_age['Sex'] = sex_age['Sex'].str[0]
sex_age.head()

Sex Age Group
0   M     35-39
1   M     35-39
2   M     35-39
3   M     35-39
4   M     35-39

In [57]:
wl_cat_total = wl_melt[['Weight Category', 'Qual Total']]
wl_tidy = pd.concat([sex_age, wl_cat_total], axis='columns')
wl_tidy.head()

Sex Age Group Weight Category  Qual Total
0   M     35-39              56         137
1   M     35-39              62         152
2   M     35-39              69         167
3   M     35-39              77         182
4   M     35-39              85         192

In [58]:
cols = ['Weight Category', 'Qual Total']
sex_age[cols] = wl_melt[cols]

In [60]:
sex_age.head()

Sex Age Group Weight Category  Qual Total
0   M     35-39              56         137
1   M     35-39              62         152
2   M     35-39              69         167
3   M     35-39              77         182
4   M     35-39              85         192

### Tidying when multiple variables are stored as column values

In [79]:
inspections = pd.read_csv('data/restaurant_inspections.csv', parse_dates=['Date'])
inspections.head(10)

Name       Date         Info  \
0  E & E Grill House 2017-08-08      Borough   
1  E & E Grill House 2017-08-08      Cuisine   
2  E & E Grill House 2017-08-08  Description   
3  E & E Grill House 2017-08-08        Grade   
4  E & E Grill House 2017-08-08        Score   
5        PIZZA WAGON 2017-12-04      Borough   
6        PIZZA WAGON 2017-12-04      Cuisine   
7        PIZZA WAGON 2017-12-04  Description   
8        PIZZA WAGON 2017-12-04        Grade   
9        PIZZA WAGON 2017-12-04        Score   

                                               Value  
0                                          MANHATTAN  
1                                           American  
2  Non-food contact surface improperly constructe...  
3                                                  A  
4                                                  9  
5                                           BROOKLYN  
6                                              Pizza  
7  Food contact surface not properly washed; rins...  
8                                                  A  
9                                                 10

In [80]:
inspections.pivot(index=['Name', 'Date'], columns='Info', values='Value')

NotImplementedError: > 1 ndim Categorical are not supported at this time

In [81]:
inspections.set_index(['Name','Date', 'Info']).head(10)

Value
Name              Date       Info                                                          
E & E Grill House 2017-08-08 Borough                                              MANHATTAN
                             Cuisine                                               American
                             Description  Non-food contact surface improperly constructe...
                             Grade                                                        A
                             Score                                                        9
PIZZA WAGON       2017-12-04 Borough                                               BROOKLYN
                             Cuisine                                                  Pizza
                             Description  Food contact surface not properly washed; rins...
                             Grade                                                        A
                             Score                                                       10

In [82]:
inspections.set_index(['Name','Date', 'Info']).unstack('Info').head()

Value  \
Info                                 Borough   
Name                   Date                    
A & L PIZZA RESTAURANT 2017-08-22   BROOKLYN   
ANTOJITOS DELI FOOD    2017-01-06   BROOKLYN   
BANGIA                 2017-06-16  MANHATTAN   
BASIL                  2017-03-05   BROOKLYN   
BIG FLEISHIG'S EXPRESS 2017-02-22   BROOKLYN   

                                                                                      \
Info                                                                         Cuisine   
Name                   Date                                                            
A & L PIZZA RESTAURANT 2017-08-22                                              Pizza   
ANTOJITOS DELI FOOD    2017-01-06  Latin (Cuban; Dominican; Puerto Rican; South &...   
BANGIA                 2017-06-16                                             Korean   
BASIL                  2017-03-05                                      Jewish;Kosher   
BIG FLEISHIG'S EXPRESS 2017-02-22                                      Jewish;Kosher   

                                                                                      \
Info                                                                     Description   
Name                   Date                                                            
A & L PIZZA RESTAURANT 2017-08-22  Facility not vermin proof. Harborage or condit...   
ANTOJITOS DELI FOOD    2017-01-06  Live roaches present in facility's food and;or...   
BANGIA                 2017-06-16  Covered garbage receptacle not provided or ina...   
BASIL                  2017-03-05  Cold food item held above 41Âº F (smoked fish ...   
BIG FLEISHIG'S EXPRESS 2017-02-22  Single service item reused; improperly stored;...   

                                               
Info                              Grade Score  
Name                   Date                    
A & L PIZZA RESTAURANT 2017-08-22     A     9  
ANTOJITOS DELI FOOD    2017-01-06     A    10  
BANGIA                 2017-06-16     A     9  
BASIL                  2017-03-05     A    13  
BIG FLEISHIG'S EXPRESS 2017-02-22     A    12

In [83]:
insp_tidy = inspections.set_index(['Name','Date', 'Info']) \
                               .unstack('Info') \
                               .reset_index(col_level=-1)
insp_tidy.head()

Value  \
Info                    Name       Date    Borough   
0     A & L PIZZA RESTAURANT 2017-08-22   BROOKLYN   
1        ANTOJITOS DELI FOOD 2017-01-06   BROOKLYN   
2                     BANGIA 2017-06-16  MANHATTAN   
3                      BASIL 2017-03-05   BROOKLYN   
4     BIG FLEISHIG'S EXPRESS 2017-02-22   BROOKLYN   

                                                         \
Info                                            Cuisine   
0                                                 Pizza   
1     Latin (Cuban; Dominican; Puerto Rican; South &...   
2                                                Korean   
3                                         Jewish;Kosher   
4                                         Jewish;Kosher   

                                                                     
Info                                        Description Grade Score  
0     Facility not vermin proof. Harborage or condit...     A     9  
1     Live roaches present in facility's food and;or...     A    10  
2     Covered garbage receptacle not provided or ina...     A     9  
3     Cold food item held above 41Âº F (smoked fish ...     A    13  
4     Single service item reused; improperly stored;...     A    12

In [84]:
insp_tidy.columns = insp_tidy.columns.droplevel(0).rename(None)
insp_tidy.head()

Name       Date    Borough  \
0  A & L PIZZA RESTAURANT 2017-08-22   BROOKLYN   
1     ANTOJITOS DELI FOOD 2017-01-06   BROOKLYN   
2                  BANGIA 2017-06-16  MANHATTAN   
3                   BASIL 2017-03-05   BROOKLYN   
4  BIG FLEISHIG'S EXPRESS 2017-02-22   BROOKLYN   

                                             Cuisine  \
0                                              Pizza   
1  Latin (Cuban; Dominican; Puerto Rican; South &...   
2                                             Korean   
3                                      Jewish;Kosher   
4                                      Jewish;Kosher   

                                         Description Grade Score  
0  Facility not vermin proof. Harborage or condit...     A     9  
1  Live roaches present in facility's food and;or...     A    10  
2  Covered garbage receptacle not provided or ina...     A     9  
3  Cold food item held above 41Âº F (smoked fish ...     A    13  
4  Single service item reused; improperly stored;...     A    12

In [85]:
inspections.set_index(['Name','Date', 'Info']) \
          .squeeze() \
          .unstack('Info') \
          .reset_index() \
          .rename_axis(None, axis='columns')

Name       Date        Borough  \
0                   A & L PIZZA RESTAURANT 2017-08-22       BROOKLYN   
1                      ANTOJITOS DELI FOOD 2017-01-06       BROOKLYN   
2                                   BANGIA 2017-06-16      MANHATTAN   
3                                    BASIL 2017-03-05       BROOKLYN   
4                   BIG FLEISHIG'S EXPRESS 2017-02-22       BROOKLYN   
5                     BLOSSOM  ON COLUMBUS 2017-01-25      MANHATTAN   
6                   BON APPETIT RESTAURANT 2017-06-27         QUEENS   
7                       BRANDY'S PIANO BAR 2017-07-24      MANHATTAN   
8                         CHARLENA CHATEAU 2017-02-28       BROOKLYN   
9                             DELI & GRILL 2017-04-26       BROOKLYN   
10                       E & E Grill House 2017-08-08      MANHATTAN   
11                             EMIR PALACE 2017-05-16       BROOKLYN   
12                  FISH MARKET RESTAURANT 2017-06-16      MANHATTAN   
13               GOURMET SPRING RESTAURANT 2017-02-13         QUEENS   
14           HAPPY TOWN CHINESE RESTAURANT 2017-03-15         QUEENS   
15                      HILLSIDE DOSA HUTT 2017-02-08         QUEENS   
16                           HOLLAND HOUSE 2017-06-02      MANHATTAN   
17                          HOLY SCHNITZEL 2017-08-31       BROOKLYN   
18                          HUNTER COLLEGE 2017-09-05      MANHATTAN   
19                 ISLAND PRIDE RESTAURANT 2017-03-17       BROOKLYN   
20  ITALIANA CREMOSA PIZZERIA & RESTAURANT 2017-08-21         QUEENS   
21                          JALISCO TACO'S 2017-09-05          BRONX   
22                   KATOU FINE RESTAURANT 2017-03-06       BROOKLYN   
23                     KAWA SUSHI 8 AVENUE 2017-12-04      MANHATTAN   
24          KING RIVER CHINESE  RESTAURANT 2017-02-15         QUEENS   
25                              LIBERTADOR 2017-02-28      MANHATTAN   
26                   MCDONALD AVENUE DINER 2017-12-06       BROOKLYN   
27                           MISTER SOFTEE 2017-06-13         QUEENS   
28              MUSIC HALL OF WILLIAMSBURG 2017-04-27       BROOKLYN   
29                                  NEGRIL 2017-01-03      MANHATTAN   
30                       NEW ASIAN CUISINE 2017-07-24         QUEENS   
31                              NU CAFE 47 2017-01-23       BROOKLYN   
32                      NY SWEET SPOT CAFE 2017-03-28       BROOKLYN   
33                PERIZIA CAFE  and  PIZZA 2017-03-27       BROOKLYN   
34                         PICCOLO'S PIZZA 2017-11-07       BROOKLYN   
35                             PIZZA WAGON 2017-12-04       BROOKLYN   
36                                  PROMDI 2017-06-13         QUEENS   
37                    PUNCH LINE JUICE BAR 2017-02-08       BROOKLYN   
38                      S & R LE JUICE BAR 2017-10-02          BRONX   
39            SAJOMA RESTAURANT & PIZZERIA 2017-02-13       BROOKLYN   
40                            SHENANIGAN'S 2017-08-16         QUEENS   
41                         THE CELTIC COVE 2017-09-08         QUEENS   
42                         VALL'S PIZZERIA 2017-03-15  STATEN ISLAND   
43                               VIP GRILL 2017-12-06       BROOKLYN   

                                              Cuisine  \
0                                               Pizza   
1   Latin (Cuban; Dominican; Puerto Rican; South &...   
2                                              Korean   
3                                       Jewish;Kosher   
4                                       Jewish;Kosher   
5                                            American   
6                                              Creole   
7                                            American   
8                                           Caribbean   
9                                        Delicatessen   
10                                           American   
11                                     Middle Eastern   
12                                            Seafood   

### Tidying when two or more values are stored in the same cell

In [86]:
cities = pd.read_csv('data/texas_cities.csv')
cities

City             Geolocation
0  Houston  29.7604° N, 95.3698° W
1   Dallas  32.7767° N, 96.7970° W
2   Austin  30.2672° N, 97.7431° W

In [87]:
geolocations = cities.Geolocation.str.split(pat='. ', expand=True)
geolocations.columns = ['latitude', 'latitude direction', 'longitude', 'longitude direction']
geolocations

latitude latitude direction longitude longitude direction
0  29.7604                  N   95.3698                   W
1  32.7767                  N   96.7970                   W
2  30.2672                  N   97.7431                   W

In [88]:
geolocations = geolocations.astype({'latitude':'float', 'longitude':'float'})
geolocations.dtypes

latitude               float64
latitude direction      object
longitude              float64
longitude direction     object
dtype: object

In [89]:
geolocations.apply(pd.to_numeric, errors='ignore')

latitude latitude direction  longitude longitude direction
0   29.7604                  N    95.3698                   W
1   32.7767                  N    96.7970                   W
2   30.2672                  N    97.7431                   W

In [90]:
cities_tidy = pd.concat([cities['City'], geolocations], axis='columns')
cities_tidy

City  latitude latitude direction  longitude longitude direction
0  Houston   29.7604                  N    95.3698                   W
1   Dallas   32.7767                  N    96.7970                   W
2   Austin   30.2672                  N    97.7431                   W

### Tidying when variables are stored in column names and values

In [92]:
sensors = pd.read_csv('data/sensors.csv')
sensors

Group     Property  2012  2013  2014  2015  2016
0     A     Pressure   928   873   814   973   870
1     A  Temperature  1026  1038  1009  1036  1042
2     A         Flow   819   806   861   882   856
3     B     Pressure   817   877   914   806   942
4     B  Temperature  1008  1041  1009  1002  1013
5     B         Flow   887   899   837   824   873

In [93]:
sensors.melt(id_vars=['Group', 'Property'], var_name='Year').head(6)

Group     Property  Year  value
0     A     Pressure  2012    928
1     A  Temperature  2012   1026
2     A         Flow  2012    819
3     B     Pressure  2012    817
4     B  Temperature  2012   1008
5     B         Flow  2012    887

In [94]:
sensors.melt(id_vars=['Group', 'Property'], var_name='Year') \
       .pivot_table(index=['Group', 'Year'], columns='Property', values='value') \
       .reset_index() \
       .rename_axis(None, axis='columns')

Group  Year  Flow  Pressure  Temperature
0     A  2012   819       928         1026
1     A  2013   806       873         1038
2     A  2014   861       814         1009
3     A  2015   882       973         1036
4     A  2016   856       870         1042
5     B  2012   887       817         1008
6     B  2013   899       877         1041
7     B  2014   837       914         1009
8     B  2015   824       806         1002
9     B  2016   873       942         1013

### Tidying when multiple observational units are stored in the same table

In [95]:
movie = pd.read_csv('data/movie_altered.csv')
movie.head()

title rating    year  duration  \
0                                      Avatar  PG-13  2009.0     178.0   
1    Pirates of the Caribbean: At World's End  PG-13  2007.0     169.0   
2                                     Spectre  PG-13  2015.0     148.0   
3                       The Dark Knight Rises  PG-13  2012.0     164.0   
4  Star Wars: Episode VII - The Force Awakens    NaN     NaN       NaN   

          director_1  director_fb_likes_1          actor_1           actor_2  \
0      James Cameron                  0.0      CCH Pounder  Joel David Moore   
1     Gore Verbinski                563.0      Johnny Depp     Orlando Bloom   
2         Sam Mendes                  0.0  Christoph Waltz      Rory Kinnear   
3  Christopher Nolan              22000.0        Tom Hardy    Christian Bale   
4        Doug Walker                131.0      Doug Walker        Rob Walker   

                actor_3  actor_fb_likes_1  actor_fb_likes_2  actor_fb_likes_3  
0             Wes Studi            1000.0             936.0             855.0  
1        Jack Davenport           40000.0            5000.0            1000.0  
2      Stephanie Sigman           11000.0             393.0             161.0  
3  Joseph Gordon-Levitt           27000.0           23000.0           23000.0  
4                   NaN             131.0              12.0               NaN

In [96]:
movie.insert(0, 'id', np.arange(len(movie)))
movie.head()

id                                       title rating    year  duration  \
0   0                                      Avatar  PG-13  2009.0     178.0   
1   1    Pirates of the Caribbean: At World's End  PG-13  2007.0     169.0   
2   2                                     Spectre  PG-13  2015.0     148.0   
3   3                       The Dark Knight Rises  PG-13  2012.0     164.0   
4   4  Star Wars: Episode VII - The Force Awakens    NaN     NaN       NaN   

          director_1  director_fb_likes_1          actor_1           actor_2  \
0      James Cameron                  0.0      CCH Pounder  Joel David Moore   
1     Gore Verbinski                563.0      Johnny Depp     Orlando Bloom   
2         Sam Mendes                  0.0  Christoph Waltz      Rory Kinnear   
3  Christopher Nolan              22000.0        Tom Hardy    Christian Bale   
4        Doug Walker                131.0      Doug Walker        Rob Walker   

                actor_3  actor_fb_likes_1  actor_fb_likes_2  actor_fb_likes_3  
0             Wes Studi            1000.0             936.0             855.0  
1        Jack Davenport           40000.0            5000.0            1000.0  
2      Stephanie Sigman           11000.0             393.0             161.0  
3  Joseph Gordon-Levitt           27000.0           23000.0           23000.0  
4                   NaN             131.0              12.0               NaN

In [97]:
stubnames = ['director', 'director_fb_likes', 'actor', 'actor_fb_likes']
movie_long = pd.wide_to_long(movie, 
                                 stubnames=stubnames, 
                                 i='id', 
                                 j='num', 
                                 sep='_').reset_index()
movie_long['num'] = movie_long['num'].astype(int)
movie_long.head(9)

id  num rating    year  duration                                     title  \
0   0    1  PG-13  2009.0     178.0                                    Avatar   
1   0    2  PG-13  2009.0     178.0                                    Avatar   
2   0    3  PG-13  2009.0     178.0                                    Avatar   
3   1    1  PG-13  2007.0     169.0  Pirates of the Caribbean: At World's End   
4   1    2  PG-13  2007.0     169.0  Pirates of the Caribbean: At World's End   
5   1    3  PG-13  2007.0     169.0  Pirates of the Caribbean: At World's End   
6   2    1  PG-13  2015.0     148.0                                   Spectre   
7   2    2  PG-13  2015.0     148.0                                   Spectre   
8   2    3  PG-13  2015.0     148.0                                   Spectre   

         director  director_fb_likes             actor  actor_fb_likes  
0   James Cameron                0.0       CCH Pounder          1000.0  
1             NaN                NaN  Joel David Moore           936.0  
2             NaN                NaN         Wes Studi           855.0  
3  Gore Verbinski              563.0       Johnny Depp         40000.0  
4             NaN                NaN     Orlando Bloom          5000.0  
5             NaN                NaN    Jack Davenport          1000.0  
6      Sam Mendes                0.0   Christoph Waltz         11000.0  
7             NaN                NaN      Rory Kinnear           393.0  
8             NaN                NaN  Stephanie Sigman           161.0

In [98]:
movie_table = movie_long[['id','title', 'year', 'duration', 'rating']]
director_table = movie_long[['id', 'director', 'num', 'director_fb_likes']]
actor_table = movie_long[['id', 'actor', 'num', 'actor_fb_likes']]

In [99]:
movie_table.head(9)

id                                     title    year  duration rating
0   0                                    Avatar  2009.0     178.0  PG-13
1   0                                    Avatar  2009.0     178.0  PG-13
2   0                                    Avatar  2009.0     178.0  PG-13
3   1  Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13
4   1  Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13
5   1  Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13
6   2                                   Spectre  2015.0     148.0  PG-13
7   2                                   Spectre  2015.0     148.0  PG-13
8   2                                   Spectre  2015.0     148.0  PG-13

In [100]:
director_table.head(9)

id        director  num  director_fb_likes
0   0   James Cameron    1                0.0
1   0             NaN    2                NaN
2   0             NaN    3                NaN
3   1  Gore Verbinski    1              563.0
4   1             NaN    2                NaN
5   1             NaN    3                NaN
6   2      Sam Mendes    1                0.0
7   2             NaN    2                NaN
8   2             NaN    3                NaN

In [101]:
actor_table.head(9)

id             actor  num  actor_fb_likes
0   0       CCH Pounder    1          1000.0
1   0  Joel David Moore    2           936.0
2   0         Wes Studi    3           855.0
3   1       Johnny Depp    1         40000.0
4   1     Orlando Bloom    2          5000.0
5   1    Jack Davenport    3          1000.0
6   2   Christoph Waltz    1         11000.0
7   2      Rory Kinnear    2           393.0
8   2  Stephanie Sigman    3           161.0

In [102]:
movie_table = movie_table.drop_duplicates().reset_index(drop=True)
director_table = director_table.dropna().reset_index(drop=True)
actor_table = actor_table.dropna().reset_index(drop=True)

In [103]:
movie_table.head()

id                                       title    year  duration rating
0   0                                      Avatar  2009.0     178.0  PG-13
1   1    Pirates of the Caribbean: At World's End  2007.0     169.0  PG-13
2   2                                     Spectre  2015.0     148.0  PG-13
3   3                       The Dark Knight Rises  2012.0     164.0  PG-13
4   4  Star Wars: Episode VII - The Force Awakens     NaN       NaN    NaN

In [104]:
director_table.head()

id           director  num  director_fb_likes
0   0      James Cameron    1                0.0
1   1     Gore Verbinski    1              563.0
2   2         Sam Mendes    1                0.0
3   3  Christopher Nolan    1            22000.0
4   4        Doug Walker    1              131.0

In [105]:
movie.memory_usage(deep=True).sum()

2289770

In [106]:
movie_table.memory_usage(deep=True).sum() + \
director_table.memory_usage(deep=True).sum() + \
actor_table.memory_usage(deep=True).sum()

2538022

In [107]:
director_cat = pd.Categorical(director_table['director'])
director_table.insert(1, 'director_id', director_cat.codes)

actor_cat = pd.Categorical(actor_table['actor'])
actor_table.insert(1, 'actor_id', actor_cat.codes)

director_table.head()

id  director_id           director  num  director_fb_likes
0   0          922      James Cameron    1                0.0
1   1          794     Gore Verbinski    1              563.0
2   2         2020         Sam Mendes    1                0.0
3   3          373  Christopher Nolan    1            22000.0
4   4          600        Doug Walker    1              131.0

In [108]:
director_associative = director_table[['id', 'director_id', 'num']]
dcols = ['director_id', 'director', 'director_fb_likes']
director_unique = director_table[dcols].drop_duplicates().reset_index(drop=True)
director_associative.head()   

id  director_id  num
0   0          922    1
1   1          794    1
2   2         2020    1
3   3          373    1
4   4          600    1

In [109]:
actor_associative = actor_table[['id', 'actor_id', 'num']]
acols = ['actor_id', 'actor', 'actor_fb_likes']
actor_unique = actor_table[acols].drop_duplicates().reset_index(drop=True)
actor_associative.head()

id  actor_id  num
0   0       824    1
1   0      2867    2
2   0      6099    3
3   1      2971    1
4   1      4536    2

In [110]:
movie_table.memory_usage(deep=True).sum() + \
director_associative.memory_usage(deep=True).sum() + \
director_unique.memory_usage(deep=True).sum() + \
actor_associative.memory_usage(deep=True).sum() + \
actor_unique.memory_usage(deep=True).sum()

1746526